In [1]:
import ROOT
import numpy as np
import utils
import stage2

%jsroot on

Welcome to JupyROOT 6.28/00


## Defining Globals

In [2]:
# TODO: add handling of angle to toml for run batches

# E_p = 10.229
# E_p = 13.79
E_p = 13.752
# E_p = 13.93
# E_p = 7.53
# E_p = 24.212
# E_p = 52.3
G_p = 0.3
hEgam_gate_FA = [6637,6702] # 127I
hEgam_gate_FE = [6150,6194] # 127I
hEgam_gate_FA_bkg = [6777,7044] # 127I

angle = np.array([36,71,72,90,108,109,144])

cos_angle = np.cos(np.pi/180*angle)
print(cos_angle)

input_file1 = "../output/stage1_output1.root"
input_file2 = "../output/stage1_output2.root"

[ 8.09016994e-01  3.25568154e-01  3.09016994e-01  6.12323400e-17
 -3.09016994e-01 -3.25568154e-01 -8.09016994e-01]


## Getting Histograms

In [3]:
# det_angle1 = stage2.get_det_angle(input_file1)
det_angle1 = [-1, 71, 90, 109, -1, -1, 71, 90, -1, 90, 90, 109, -1, 71, -1, 90, -1, 108, 144, 72, 36, -1, -1, -1, -1, 36, 144, -1, 90, -1, -1, -1]

hEn1 = stage2.get_hist(input_file1,"hEn_gate_FA_beamcorr_pucorr")
hEn_bkg1 = stage2.get_hist(input_file1,"hEn_gate_FA_bkg_beamcorr_pucorr")
hEgam1 = stage2.get_hist(input_file1,"hEgam")

# det_angle2 = stage2.get_det_angle(input_file2)
det_angle2 = [-1, 71, 90, 109, -1, -1, 71, 90, -1, 90, 90, 109, -1, 71, -1, 90, -1, 108, 144, 36, -1, -1, -1, -1, -1, 36, 144, -1, 90, -1, -1, -1]

hEn2 = stage2.get_hist(input_file2,"hEn_gate_FA_beamcorr_pucorr")
hEn_bkg2 = stage2.get_hist(input_file2,"hEn_gate_FA_bkg_beamcorr_pucorr")
hEgam2 = stage2.get_hist(input_file2,"hEgam")

## Performing Background Subtractions

In [4]:
hEgam_bkg1 = stage2.calc_hEgam_bkg(hEgam1,6500,7100)
scale1 = stage2.calc_scale(hEgam_bkg1, hEgam_gate_FA, hEgam_gate_FA_bkg)
hEn_sub1 = stage2.calc_hEn_sub(hEn1, hEn_bkg1, scale1)

hEgam_bkg2 = stage2.calc_hEgam_bkg(hEgam2,6500,7100)
scale2 = stage2.calc_scale(hEgam_bkg2, hEgam_gate_FA, hEgam_gate_FA_bkg)
hEn_sub2 = stage2.calc_hEn_sub(hEn2, hEn_bkg2, scale2)

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


## Calculating N_L and N_H

In [5]:
N_L_det1, N_H_det1 = stage2.calc_N_LH(hEn_sub1, E_p, G_p)

N_L_det2, N_H_det2 = stage2.calc_N_LH(hEn_sub2, E_p, G_p)

## Adding N_L and N_H By Angle

In [6]:
N_L_angle1 = stage2.sort_by_angle(N_L_det1,det_angle1)
N_H_angle1 = stage2.sort_by_angle(N_H_det1,det_angle1)
N_L_angle2 = stage2.sort_by_angle(N_L_det2,det_angle2)
N_H_angle2 = stage2.sort_by_angle(N_H_det2,det_angle2)

## Adding Runs

In [7]:
N_L_angle = N_L_angle1 + N_L_angle2 
N_H_angle = N_H_angle1 + N_H_angle2 

# N_L_angle = N_L_angle1 
# N_H_angle = N_H_angle1 

print(N_H_angle)
print(N_L_angle)



[ 489.00325936  525.79396189  227.46287224 1326.28758426  140.43548064
  741.26954842  835.27594066]
[ 541.44313147  550.81844523  237.81663688 1275.40479461  127.609981
  437.65675506  592.83188152]


## Calculating A_LH and dA_LH

In [8]:
A_LH_angle = stage2.A_LH(N_L_angle,N_H_angle)
dA_LH_angle = stage2.dA_LH(N_L_angle,N_H_angle)

print(dA_LH_angle)

# A_LH_angle1 = stage2.A_LH(N_L_angle1,N_H_angle1)
# dA_LH_angle1 = stage2.dA_LH(N_L_angle1,N_H_angle1)

# A_LH_angle2 = stage2.A_LH(N_L_angle2,N_H_angle2)
# dA_LH_angle2 = stage2.dA_LH(N_L_angle2,N_H_angle2)


[0.03111173 0.03046863 0.04634848 0.01960148 0.06100958 0.028142
 0.0260777 ]


In [9]:
# graph = ROOT.TGraphErrors(len(A_LH_angle[1:]),cos_angle[1:], A_LH_angle[1:], np.zeros_like(A_LH_angle[1:]), dA_LH_angle[1:])
# graph.SetMarkerStyle(20)
# graph.GetXaxis().SetTitle("cos#theta_{#gamma}")
# graph.GetYaxis().SetTitle("A_{LH}")
# # graph.SetTitle(title)

# canvas = ROOT.TCanvas("canvas", "My Graph", 600, 500)
# graph.Draw()
# fitFunc = ROOT.TF1("fitFunc", "[0]*x + [1]")
# graph.Fit(fitFunc, "Q")
# ROOT.gStyle.SetOptFit(1)
# graph.Draw("AP")
# canvas.Draw()
A_LH_graph = stage2.create_A_LH_graph(cos_angle, A_LH_angle, dy = dA_LH_angle)
stage2.linear_fit_and_plot(A_LH_graph)